In [10]:
# React Agent
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
import requests
from langgraph_swarm import create_swarm
from typing import Tuple, List
import csv

MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def get_weather(city_name: str) -> str:
    """
    Fetches comprehensive weather information for a specified city using the wttr.in service.

    Parameters:
    city_name (str): The name of the city for which to fetch weather information.

    Returns:
    str: A string containing detailed weather information including weather condition, temperature,
         wind speed, humidity, precipitation, pressure, moon phase, maximum temperature, and sunrise/sunset times.

    Example usage:
    city_name = "Makati"
    weather_data = get_weather(city_name)
    print(weather_data)
    """
    weather_url = f"https://wttr.in/{city_name}?format=%C+%t+%w+%h+%p+%P+%m+%M+%S"
    response = requests.get(weather_url)
    weather_data = response.text

    print("Weather Agent used")
    return weather_data


weather_agent = create_react_agent(
    model=MODEL,
    tools=[get_weather],
    prompt = """
    check the weather using your tool
    """,
    name = "weather_agent",
    checkpointer=checkpointer
)

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )


    print("Currency agent used")
    return response.json()

currency_agent = create_react_agent(
    model=MODEL,
    tools=[get_exchange_rate],
    prompt = """
    get the exchange rate using your tool
    """,
    name = "exchange_rate_agent",
    checkpointer=checkpointer
)

boss = create_supervisor(
    model=MODEL,
    agents =[weather_agent, currency_agent],
    prompt = """
    You are a supervisor agent. You interact with the user directly and help the user for queries regarding weather and currency.

    For weather concerns, you'll consult the weather_agent.
    For currency related concerns, you'll consult the currency_agent.

    Only you and the user can interact, the user can't see the messages of the other agents, so relay the answer of the agents to the user. Always end your answer with Mwah!
    """,
    supervisor_name = "Bossing",
).compile(checkpointer=checkpointer)


swarms = create_swarm([weather_agent, currency_agent], default_active_agent="weather_agent").compile(checkpointer=checkpointer)

Weather Chat Bot

In [26]:
#weather
config = {"configurable": {"thread_id": "1"}}
response = weather_agent.invoke(
    {"messages":[{"role": "user", "content": "what is the exchange rate of USD to PHP?"}]},
    config,
)

print(response['messages'][-1].content)

I am sorry, I cannot fulfill this request. The available tools do not have the ability to provide exchange rates. I can only get weather information for a specific city.



Currency Chat Bot

In [53]:
#currency
config = {"configurable": {"thread_id": "1"}}
response = currency_agent.invoke(
    {"messages":[{"role": "user", "content": "what is the exchange rate of USD to PHP?"}]},
    config,
)

print(response['messages'][-1].content)

I do not have the capability to search for weather information.



Supervisor Chat Bot

In [3]:
#supervisor
config = {"configurable": {"thread_id": "1"}}
response = boss.invoke(
    {"messages":[{"role": "user", "content": "What is the weather in Santa Rosa, Laguna Province?"}]},
    config,
)

print(response['messages'][-1].content)

Weather Agent used
The weather in Santa Rosa, Laguna Province is partly cloudy with a temperature of 17°C, wind speed of 6km/h, humidity of 88%, precipitation of 0.0mm, pressure of 1019hPa, and a waning gibbous moon. The maximum temperature will be 24°C and sunrise will be at 05:42:55. Mwah!



In [4]:
response

{'messages': [HumanMessage(content='What is the weather in Santa Rosa, Laguna?', additional_kwargs={}, response_metadata={}, id='7a9d47b0-07ad-4494-bb19-88e2e7d17dcf'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city_name": "Santa Rosa, Laguna"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 142, 'candidates_token_count': 10, 'total_token_count': 152, 'prompt_tokens_details': [{'modality': 1, 'token_count': 142}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 10}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.01170167252421379, 'model_name': 'gemini-2.0-flash-001'}, name='weather_agent', id='run-6af3d059-0da1-4150-a36c-937e248bbc43-0', tool_calls=[{'name': 'get_weather', 'args': {'city_name': 'Santa Rosa, Laguna'}, 'id': '62a3579a-c93d-4a41-bfb4-ef656ecadba3', 'type': 'tool_call'}], usage_metadat

In [47]:
graph = swarms.get_graph()
print(graph)
print("Graph:", graph) 

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'weather_agent': Node(id='weather_agent', name='weather_agent', data=<langgraph.graph.state.CompiledStateGraph object at 0x000002162102AD90>, metadata=None), 'exchange_rate_agent': Node(id='exchange_rate_agent', name='exchange_rate_agent', data=<langgraph.graph.state.CompiledStateGraph object at 0x0000021620E97250>, metadata=None)}, edges=[Edge(source='__start__', target='weather_agent', data=None, conditional=True), Edge(source='__start__', target='exchange_rate_agent', data=None, conditional=True)])
Graph: Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'weather_agent': Node(id='weather_agent', name='weather_agent', data=<langgraph.graph.state.CompiledStateGraph object at 0x000002162102AD90>, metadata=None), 'exchange_rate_agent': Node(id='exchange_rat

Activity 3: Create an agent that would recommend you the best resorts today
 
The agent has two tools, one for checking the weather, the next one is for checking a list of resorts.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
import requests
from typing import Tuple, List
import csv

MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def get_weather(city_name: str) -> str:
    """
    Fetches comprehensive weather information for a specified city using the wttr.in service.
    """
    weather_url = f"https://wttr.in/{city_name}?format=%C+%t+%w+%h+%p+%P+%m+%M+%S"
    response = requests.get(weather_url)
    if response.status_code != 200:
        return "Weather information unavailable at the moment."
    
    print("Weather Agent used")
    return response.text


def get_resorts(city_name: str, csv_path: str = "C:/Users/richard.c.ferrer/Agents/resorts_data.csv") -> List[List[str]]:
    """
    Retrieves resorts from a CSV file filtered by the specified city name.

    Returns:
    List of [Resort Name, Location]
    """
    resorts = []
    try:
        with open(csv_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if city_name.lower() in row["Location"].lower():
                    resorts.append([row["Resort"], row["Location"]])
    except FileNotFoundError:
        print("CSV file not found.")
    except KeyError:
        print("CSV headers are incorrect. Expected: 'Resort' and 'Location'.")

    return resorts

def get_bestresorts(city_name: str) -> Tuple[str, List[List[str]]]:
    """
    Combines weather data and resort list into a single tool response.
    """
    weather_data = get_weather(city_name)
    resorts_data = get_resorts(city_name)
    return weather_data, resorts_data

bestresorts_agent = create_react_agent(
    model=MODEL,
    tools=[get_bestresorts],
    prompt="""
    Check the weather and get the best resorts using your tool.
    """,
    name="bestresorts_agent",
    checkpointer=checkpointer
)



In [22]:
response = bestresorts_agent.invoke(
    {
        "messages": [{"role": "user", "content": "What are some good resorts in Calamba?"}]
    },
    config={"configurable": {"thread_id": "1"}},
)
print(response['messages'][-1].content)


Weather Agent used
The weather in Calamba is Sunny +36°C, with wind from the West-Southwest at 14km/h.

Here are some resorts in Calamba: The Cliffhouse Calamba, Casa Palmera Royale, La Vista Pansol Resort, 88 Hotspring Resort, and Hannah Mountain Resort and Events Place.



In [ ]:
# React Agent
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
import requests
from langgraph_swarm import create_swarm
from typing import Tuple, List
import csv

MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def get_weather(city_name: str) -> str:
    """
    Fetches comprehensive weather information for a specified city using the wttr.in service.

    Parameters:
    city_name (str): The name of the city for which to fetch weather information.

    Returns:
    str: A string containing detailed weather information including weather condition, temperature,
         wind speed, humidity, precipitation, pressure, moon phase, maximum temperature, and sunrise/sunset times.

    Example usage:
    city_name = "Makati"
    weather_data = get_weather(city_name)
    print(weather_data)
    """
    weather_url = f"https://wttr.in/{city_name}?format=%C+%t+%w+%h+%p+%P+%m+%M+%S"
    response = requests.get(weather_url)
    weather_data = response.text

    print("Weather Agent used")
    return weather_data


weather_agent = create_react_agent(
    model=MODEL,
    tools=[get_weather],
    prompt = """
    check the weather using your tool
    """,
    name = "weather_agent",
    checkpointer=checkpointer
)

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )


    print("Currency agent used")
    return response.json()

currency_agent = create_react_agent(
    model=MODEL,
    tools=[get_exchange_rate],
    prompt = """
    get the exchange rate using your tool
    """,
    name = "exchange_rate_agent",
    checkpointer=checkpointer
)

boss = create_supervisor(
    model=MODEL,
    agents =[weather_agent, currency_agent],
    prompt = """
    You are a supervisor agent. You interact with the user directly and help the user for queries regarding weather and currency.

    For weather concerns, you'll consult the weather_agent.
    For currency related concerns, you'll consult the currency_agent.

    Only you and the user can interact, the user can't see the messages of the other agents, so relay the answer of the agents to the user. Always end your answer with Mwah!
    """,
    supervisor_name = "Bossing",
).compile(checkpointer=checkpointer)


swarms = create_swarm([weather_agent, currency_agent], default_active_agent="weather_agent").compile(checkpointer=checkpointer)

Acitivity 4 Shopping AI Assistant

In [2]:
# React Agent
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
from langgraph_swarm import create_swarm

# Model setup
MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,
}
MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

# 🛍️ Fashion Knowledgebase Tool
def fashion_knowledgebase(query: str) -> str:
    """
    A tool that provides product recommendations in the fashion category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "dress": "Summer Floral Dress - $49",
        "jacket": "Denim Jacket - $89",
        "shoes": "Classic White Sneakers - $69"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    return "Sorry, I only know about dresses, jackets, and shoes."

# 📱 Gadget Knowledgebase Tool
def gadget_knowledgebase(query: str) -> str:
    """
    A tool that provides product recommendations in the gadgets category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "laptop": "14-inch Ultrabook - $899",
        "smartphone": "Flagship Phone X - $999",
        "earbuds": "Noise-Canceling Earbuds - $199"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    return "Sorry, I only know about laptops, smartphones, and earbuds."

# 🧠 Create React Agent for Fashion
fashion_agent = create_react_agent(
    model=MODEL,
    tools=[fashion_knowledgebase],
    name="fashion_agent_unique",
    prompt="You are a fashion product recommender. Only use your tool to recommend fashion items.",
    checkpointer=checkpointer
)

# 🧠 Create React Agent for Gadgets
gadget_agent = create_react_agent(
    model=MODEL,
    tools=[gadget_knowledgebase],
    name="gadget_agent_unique",
    prompt="You are a gadget assistant. Recommend only from your tool.",
    checkpointer=checkpointer
)


# 🧑‍💼 Supervisor Agent (Unified Persona)
boss = create_supervisor(
    model=MODEL,
    agents=[fashion_agent, gadget_agent],
    prompt="""
You are Bossing, a helpful shopping assistant. You help users shop for two categories only: fashion and gadgets.

If the user asks about clothing (e.g., dresses, shoes, jackets), respond using the fashion_agent.
If the user asks about electronics (e.g., laptops, smartphones, earbuds), respond using the gadget_agent.

You must not suggest products outside of these categories or tools.
Do not mention agents or swarms. The user sees you as one assistant.

Always end your reply with: Happy shopping! 🛍️
""",
    supervisor_name="Bossing_unique",
).compile(checkpointer=checkpointer)

# ✅ Run test conversation
response = boss.invoke(
    {
        "messages": [
            {"role": "user", "content": "I'm looking for a jacket."},

        ]
    },
    config = {"configurable": {"thread_id": "1"}},
)

print(response['messages'][-1].content)



Happy shopping! 🛍️



working

In [3]:
# Shopping AI Assistant
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
import requests
from langgraph_swarm import create_swarm
from typing import Tuple, List
import csv

MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def fashion_data(query: str) -> str:
    """
    A tool that provides product recommendations in the fashion category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "dress": "Summer Floral Dress - $49",
        "jacket": "Denim Jacket - $89",
        "shoes": "Classic White Sneakers - $69"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    return "Sorry, I only know about dresses, jackets, and shoes."


fashion_agent = create_react_agent(
    model=MODEL,
    tools=[fashion_data],
    name = "fashion_agent",
    prompt = """
    You are a fashion product recommender. Only use your tool to recommend fashion items.
    """,
    checkpointer=checkpointer
)

def gadget_data(query: str) -> str:
    """
    A tool that provides product recommendations in the gadgets category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "laptop": "14-inch Ultrabook - $899",
        "smartphone": "Flagship Phone X - $999",
        "earbuds": "Noise-Canceling Earbuds - $199"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    return "Sorry, I only know about laptops, smartphones, and earbuds."

gadget_agent = create_react_agent(
    model=MODEL,
    tools=[gadget_data],
    name = "gadget_agent",
    prompt = """
    You are a gadget assistant. Recommend only from your tool.
    """,
    checkpointer=checkpointer
)

boss = create_supervisor(
    model=MODEL,
    agents =[fashion_agent, gadget_agent],
    prompt = """
    You are Bossing, a helpful shopping assistant. You help users shop for two categories only: fashion and gadgets.

If the user asks about clothing (e.g., dresses, shoes, jackets), respond using the fashion_agent.
If the user asks about electronics (e.g., laptops, smartphones, earbuds), respond using the gadget_agent.

You must not suggest products outside of these categories or tools.
Do not mention agents or swarms. The user sees you as one assistant.

Always end your reply with: Happy shopping! 🛍️
    """,
    supervisor_name = "Bossing",
).compile(checkpointer=checkpointer)



In [6]:
#supervisor
config = {"configurable": {"thread_id": "1"}}
response = boss.invoke(
    {"messages":[{"role": "user", "content": input()}]},
    config,
)

print(response['messages'][-1].content)

I do not have information about that topic. I can only help with fashion and gadgets.



try

In [ ]:
# Shopping AI Assistant
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
from langgraph_swarm import create_swarm, create_handoff_tool


MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def customer_service(query: str) -> str:
    """
    A tool that provides customer service responses.
    It returns a greeting message if the user greets, otherwise raises an exception.
    """
    greetings = ["hi", "hello", "good day", "hey"]
    if any(greet in query.lower() for greet in greetings):
        return "Thank you for reaching out! How can I assist you today?"
    # If not a greeting, raise Exception to trigger handoff
    raise Exception("Not a greeting.")

csr_agent = create_react_agent(
    model=MODEL,
    tools=[customer_service,create_handoff_tool(
            agent_name="Fashionista",
            description="Transfer to Fashionista for fashion-related queries (dress, jacket, shoes)."
        ),
        create_handoff_tool(
            agent_name="Gadgenista",
            description="Transfer to Gadgenista for gadget-related queries (laptop, smartphone, earbuds)."
        )
    ],
    name = "Customerista",
    prompt = """
    You are Customerista, a customer service representative. Greet the user if they say hi, hello, or similar. Otherwise, hand off to the right agent.
    """,
    checkpointer=checkpointer
)

def fashion_data(query: str) -> str:
    """
    A tool that provides product recommendations in the fashion category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "dress": "Summer Floral Dress - $49",
        "jacket": "Denim Jacket - $89",
        "shoes": "Classic White Sneakers - $69"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    # return "Sorry, I only know about dresses, jackets, and shoes."
    # If not a fashion query, raise an exception to let the swarm try the next agent
    raise Exception("Not a fashion query.")



fashion_agent = create_react_agent(
    model=MODEL,
    tools=[fashion_data, create_handoff_tool(
        agent_name="Gadgenista",
        description = "Transfer to Gadgenista, It can help you with gadgets such as a laptop, smartphone, and earbuds.",
        )
    ],
    name = "Fashionista",
    prompt = """
    You are Fashionista, fashion product recommender. Only use your tool to recommend fashion items.
    """,
    checkpointer=checkpointer
)

def gadget_data(query: str) -> str:
    """
    A tool that provides product recommendations in the gadgets category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "laptop": "14-inch Ultrabook - $899",
        "smartphone": "Flagship Phone X - $999",
        "earbuds": "Noise-Canceling Earbuds - $199"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    # return "Sorry, I only know about laptops, smartphones, and earbuds."
    raise Exception("Not a gadget query.")

gadget_agent = create_react_agent(
    model=MODEL,
    tools=[gadget_data, create_handoff_tool(agent_name="Fashionista",
        description = "Transfer to Fashionista, It can help you with fashion such as a dress, jacket, and shoes.",
        )
    ],
    name = "Gadgenista",
    prompt = """
    You are Gadgenista, a gadget assistant. Recommend only from your tool.
    """,
    checkpointer=checkpointer
)


swarms = create_swarm(
    [csr_agent, fashion_agent, gadget_agent],
    default_active_agent="Customerista",
).compile(checkpointer=checkpointer)


boss = create_supervisor(
    model=MODEL,
    agents =[fashion_agent, gadget_agent],
    prompt = """
    You are Bossing, a helpful shopping assistant. You help users shop for two categories only: fashion and gadgets.

If the user asks about clothing (e.g., dresses, shoes, jackets), respond using the fashion_agent.
If the user asks about electronics (e.g., laptops, smartphones, earbuds), respond using the gadget_agent.

You must not suggest products outside of these categories or tools.
Do not mention agents or swarms. The user sees you as one assistant.

Always end your reply with: Happy shopping! 🛍️
    """,
    supervisor_name = "Bossing"
).compile(checkpointer=checkpointer)



In [ ]:
#supervisor
config = {"configurable": {"thread_id": "1"}}
response = boss.invoke(
    {"messages":[{"role": "user", "content": "Good day!"}]},
    config,
)

print(response['messages'][-1].content)

In [ ]:
#swarms
config = {"configurable": {"thread_id": "1"}}
response = swarms.invoke(
    {"messages":[{"role": "user", "content": "Hello? I want a dress"}]},
    config,
)

print(response['messages'][-1].name)
print('----------------------------------------')
print(response['messages'][-1].content)

Fashionista
----------------------------------------
I recommend a Summer Floral Dress for $49.

